# Chapter 3: Dictionaries and Sets

The `dict` type is a fundamental part of Python impementation. Module namespaces, class and instance attributes and function keyword arguments are examples of where dictionaries are deployed.

Dictionaries are highly optimised, powered by hash tables.

## Generic Mapping Types

The `collections.abc` module provides the `Mapping` and `MutableMapping` ABCs to formalise the interfaces of `dict` and similar types (see **p. 68** for details).

Implementations of specialised mapping often extend `dict` or `collections.UserDict` instead of these ABCs.

All mapping types in the standard library require that the keys must be hashable.

## `dict` Comprehensions

Dictionary comprehension (*dictcomp*) builds a `dict` instance by producing `key:value` pairs from any iterable.

In [3]:
DIAL_CODES = {
    (86, "China"),
    (91, "India"),
    (1, "United States"),
}

{code: country for code, country in DIAL_CODES}

{91: 'India', 1: 'United States', 86: 'China'}

In [4]:
{country: code for code, country in DIAL_CODES}

{'India': 91, 'United States': 1, 'China': 86}

In [5]:
{country.upper(): code for code, country in DIAL_CODES}

{'INDIA': 91, 'UNITED STATES': 1, 'CHINA': 86}

## Overview of Common Mapping Methods

See **p. 71** for common mapping methods implemented by `dict` and two of its mostt powerful variations, `defaultdict` and `OrderedDict`.

Noteworthy methods are `dict.update(m, [**kwargs])` and `dict.setdefault(k, [default)`. 

The former is used internally by many constructors of Python mappings. `update` first checks if `m` has the `.keys()` method. If it does, it assumes it is a mapping. Otherwise, `update` iterates over `m`, assuming its items are `(key, value)` pairs.

In [9]:
iter_update = [(47, "Norway")]
country_codes.update(iter_update)
country_codes

{91: 'India', 1: 'United States', 86: 'China', 47: 'Norway'}

The latter provides a significant speedup by avoiding redundant key lookups.

### Handling Missing Keys with `setdefault`

In line with Python's *fail-fast philosophy*, `d[k]` wil throw a `KeyError` if `k` is not an existing key.

An alternative is to use `d.get(k, [default])`, which avoids the `KeyError` exception and returns default if `k` is not an existing key. However, this method can get slow and inefficient to work with when updating the value (if mutable), if the key is found.

An alternative is to use `d.setdefault(k, [default])`.

In [ ]:
# For some key k and value v
# Inefficient
value = dict.get(k, []) # First search through dict
value.update(v)
d[k] = value # Second search through dict

# Efficient
dict.setdefault(k, []).append(v) # Only one search through dict

### Mappings with Flexible Key Lookup

Sometimes it is convenient to have mappings that return some made-up value when a missing key is search. There are two approaches:

* `collections.defaultdict`

* subclass `dict` (or any other mapping type) and add `__missing__` methods

#### `defaultdict`

A `defaultdict` is cofigured to create items on demand whenever a missing key is searched.

When instantiating, provide a callable that is used to produce a default value whenever `__getitem__` is passed a non-existant key argument. The callable is stored in an instance attribute called `default_factory`.

In [13]:
from collections import defaultdict

dd = defaultdict(list)
dd["key"]
dd

defaultdict(list, {'key': []})

#### The `__missing__` method

This method is not defined in the base `dict` class, but `dict` is aware of it. If `dict` is subclassed and a `__missing__` method is provided, the standard `dict.__getitem__` will call it whenever a key is not found (instead of returning `KeyError`). See **pp. 76-77** for an example.

### Variations of `dict`

#### `collections.OrderedDict`

Maintains key in insertion order, allowing predictable iteration. `popitem` method pops the last item by default, unless argument `last=False` in which case the first item is popped.

#### `collections.ChainMap`

See **p. 79**.

#### `collections.Counter`

A mapping that holds an integer count for each key. Implements `+`, `-` and `most_common([n])` for convenient manipulation

In [18]:
from collections import Counter

string = "abracadabra"
counter = Counter(string)
counter

Counter({'a': 5, 'b': 2, 'r': 2, 'c': 1, 'd': 1})

In [19]:
counter.update("aaaaaazzzz")
counter

Counter({'a': 11, 'b': 2, 'r': 2, 'c': 1, 'd': 1, 'z': 4})

In [20]:
counter.most_common(2)

[('a', 11), ('z', 4)]

#### `collections.UserDict`

A pure Python implementation of a mapping that works like a standard `dict`.

### Subclassing `UserDict`

`UserDict` should be preferred over `dict` when subclassing, because `dict` has some implementation shortcuts that forces us to override methods that can be inherited from `UserDict` with no problems (see example 3-7 and 3-8 on **pp. 77-81**).

`UserDict` inherits from `MutableMapping`, which has some particularly useful methods such as `update` and `get` (see **p. 81**).

## Immutable Mappings

`types.MappingProxyType` returns an instance `mappingproxy` that is a dynamic view of the original mapping, but read-only. This prevents the user from inadvertantly changing a mapping by mistake.

In [30]:
from types import MappingProxyType

d = {}
d[1] = "A"
d_proxy = MappingProxyType(d)
d_proxy

mappingproxy({1: 'A'})

In [31]:
d_proxy[2] = "B"

TypeError: 'mappingproxy' object does not support item assignment

In [32]:
d[2] = "B"
d_proxy

mappingproxy({1: 'A', 2: 'B'})

## Set Theory

A set is a collection of unique objects, and in the following "set" refers to both `set` and `frozenset`.

Set elements must be hashable. The `set` type is not hashable, but `frozenset` is. `frozenset` elements can be included in a `set`.

The infix operators `|`, `&` and `-` compute the union, intersection and difference, respectively.

### `set` Literals

The syntax of `set` literals looks exactly like math notation - `{1, 2}` - with the exception of empty sets. In the case of an empty set, we must use `set()`.

`frozenset` must be created by calling the constructor, e.g. `frozenset({1, 2, 3})`.

### `set` Comprehensions

We can use *set comprehensions* (setcomps) to build sets, just like with dictcomps and listcomps.

### `set` Operations

The infix operaters require that both operands be `set` types.

All other methods can take one or more iterables as an argument.

See table 3-2 on **p. 88** for a complete overview.

In [41]:
a = {1, 2, 3, 4, 5}
b = {1, 4, 5}
c = [5, 7, 8]

a & b

{1, 4, 5}

In [42]:
a & c

TypeError: unsupported operand type(s) for &: 'set' and 'list'

In [43]:
a.intersection(b, c)

{5}

`set` predicates are also available, and allow the user to check for e.g. subsets, supersets, etc.

See table 3-3 on **p. 89** for a complete overview of these and additional `set` methods (table 3-4).

In [49]:
# a is a superset of b
a >= b

# b is a subset of a
b <= a

# b is a proper subset of a
b < a

True

## `dict` and `set` Under the Hood

Python `set` and `dict` types are implemented using hash tables.

A hash table is a sparse array (i.e. an array that always has empty cells).

The cells in a hash table are often called "buckets".

In a `dict` hash table, there is a bucket for each item containing two fields: A reference to the key and a reference to the value of the item. All buckets have the same size, and access to an individual bucket is done by offset (index).

Python truies to keep at least 1/3 of the buckets empty. If the hash table becomes too crowded, Python wil create a new one.

To put an item in a hash table, first a *hash value* must be calculated (done with the built-in `hash` function).

If to objects are equal, their hash must also be equal.

Furthermore, hash values should scatter around the index space as much as possible (i.e. similar values can have wildly different hashes).

For details on Python's hash algorithm, see **p. 94**. Summarised, the value at `my_dict[search_key]` is found by:

1. Calling `hash(search_key)` to determine hash value

    a. If found bucket is empty, return `KeyError`
    
    b. Otherwise, a `found_key:found_value` pair is found
    
    
2. If 1.b., check if `search_key == found_key`
   
    a. If match, return `found_value`
    
    b. If `search_key != found_key`, we have a *hash collision*
    
    
3. If 3.b., resolve hash collision.

    a. If bucket is empty, return `KeyError`
    
    b. If the bucket is not empty, return `found_value`
    
    c. If none of the above, collision resolution process is repeated
    
The process to update or insert an item is the same, except that when an empty bucket is located, the new item is put there. When a bucket with a matching key is found, the value is overwritten.

If the hash table is too crowded upon insertion, Python rebuilds it with more room.

### Practical Consequences of How `dict` Works

**Keys must be hashable objects**, which requires:

1. It supports the `hash` function via a `__hash__` method thatt always returns the same value over the lifetime of the object.

2. It supports equality via an `__eq__` method.

3. If `a == b` is `True`, the `hash(a) == hash(b)` must also be `True`.

*Note:* If a custom `__eq__` is implemented, a suitable `__hash__` must also be impemented to ensure point 3. in the above list.

**`dicts` have significant memory overhead**, because the required sparse arrays are not space efficient.

**Key search is very fast**, regardless of the size of the hash table.

**Key ordering depends on insertion order**. If a hash collision occurs, a key ends up in a position it would not normally occupy. This means that two `dict` types could compare equal even though their key orderings are different.

**Adding items to a dict may change the order of the existing keys**. Because hash tables can be rebuilt, we cannot reliably predict when new hash collision might happen when adding items to the new table.

*Note:* This is the reason why we should never modify the contents of a `dict` while iterating over it. Instead, read `dict` from start to finish and collect needed additions in a second `dict`. Then update first `dict` with contents of second `dict`.

The `set` type is also implemented with hash tables, but each bucket contains only a reference to an element. Everything above also applies to `set`:

* Set elements must be hashable.

* Sets have significant memory overhead.

* Membership testing is very efficient.

* Element ordering depends on insertion order.

* Adding elements to a set may change the order of other elements.